# Part of Speech tagging

Authors:
* Konstantin Taranov (exercise code and descriptions)
* Ondrej Skopek (rewrite to a Jupyter Notebook)

## Imports and visualization functions

In [1]:
# Install the libraries if not already on your system:

# Without virtualenv:
!pip install --user nltk numpy tabulate

# In a virtualenv:
# !pip install nltk numpy tabulate  

In [2]:
import nltk
import numpy as np
import ssl

# Solves the problem with SSL certificates while downloading nltk packets
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('tagsets')

from IPython.display import HTML, display, Markdown
import tabulate

def md(*md):
    """Displays rendered Markdown."""
    display(Markdown(" ".join(map(str, md))))

def show_tags(tags):
    """Displays an HTML table for a tagged sentence."""
    tags = list(map(list, zip(*tags)))
    display(HTML(tabulate.tabulate(tags, tablefmt='html')))
    
def show_viterbi(viterbi, tags, sentence, t):
    """Displays an HTML table for a Viterbi matrix."""
    md("#### Viterbi probabilities at time $t = {}$".format(t))
    viterbi = viterbi.tolist()
    for tag_idx, row in enumerate(viterbi):
        new_row = [tags[tag_idx]]
        for sent_idx, element in enumerate(row):
            new_row.append("0" if element == 0 else "{:.2e}".format(element))
        viterbi[tag_idx] = new_row
    viterbi.insert(0, [""] + sentence)
    
    display(HTML(tabulate.tabulate(viterbi, tablefmt='html')))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wangr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\wangr\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\wangr\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\wangr\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


## PoS tagging

PoS tagging is the process of tagging a word in a text where the tag is a particular part of speech mark.

The 12 universal tags are:

* `VERB` - verbs (all tenses and modes)
* `NOUN` - nouns (common and proper)
* `PRON` - pronouns
* `ADJ` - adjectives
* `ADV` - adverbs
* `ADP` - adpositions (prepositions and postpositions)
* `CONJ` - conjunctions
* `DET` - determiners
* `NUM` - cardinal numbers
* `PRT` - particles or other function words
* `X` - other: foreign words, typos, abbreviations
* `.` - punctuation

### Examples of tagging:

In [3]:
text = nltk.word_tokenize("ETH is the best university in the world.")
tags = nltk.pos_tag(text,tagset='universal')
show_tags(tags)

ETH,is,the,best,university,in,the,world,.
NOUN,VERB,DET,ADJ,NOUN,ADP,DET,NOUN,.


In [4]:
text = nltk.word_tokenize("Dogs are animals.")
tags = nltk.pos_tag(text,tagset='universal')
show_tags(tags)

Dogs,are,animals,.
NOUN,VERB,NOUN,.


## Implementing your own PoS tagger

In this exercise, you should implement your own PoS tagger using a Hidden Markov Model.

For learning your model the following Tagged Corpora can be used:

In [5]:
# nltk.download('treebank')
#from nltk.corpus import treebank as corpus

nltk.download('brown')
from nltk.corpus import brown as corpus

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\wangr\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


First, we add an artificial `START` and `END` tag at the beginning and end of each sentence.

In [6]:
tagged_words = []
all_tags = []

for sent in corpus.tagged_sents(tagset='universal'):
    tagged_words.append(("START", "START"))
    all_tags.append("START")
    for word, tag in sent:
        all_tags.append(tag)
        tagged_words.append((tag, word))
    tagged_words.append(("END", "END"))
    all_tags.append("END")

### Estimating probabilities
For the Viterbi algorithm we need to compute:

1. The maximum likelihood estimate of a transition probability

$$P(t_{i} | t_{i-1}) = \frac{C(t_{i-1}, t_{i})}{C(t_{i-1})}$$

2. Emission probabilities

$$P(w_{i} | t_{i}) =  \frac{C(t_{i}, w_{i})}{C(t_{i})}$$

#### Transitions

Compute $C(t_{i-1}, t_{i})$

In [7]:
cfd_tags= nltk.ConditionalFreqDist(nltk.bigrams(all_tags))

Transition probabilities $P(t_{i} | t_{i-1})$

In [8]:
cpd_tags = nltk.ConditionalProbDist(cfd_tags, nltk.MLEProbDist)

##### Examples

In [9]:
# C('DET', 'NOUN'):
print("Frequency of C('DET', 'NOUN') is",
      cfd_tags['DET']['NOUN'] )

# P('NOUN' | 'DET')
print("Probability of P('NOUN' | 'DET') is",
      cpd_tags['DET'].prob('NOUN') )

Frequency of C('DET', 'NOUN') is 85838
Probability of P('NOUN' | 'DET') is 0.6264678621213117


#### Emmision

Compute $C(t_{i}, w_{i})$

In [10]:
cfd_tagwords = nltk.ConditionalFreqDist(tagged_words)

Emission probabilities $P(w_{i} | t_{i})$

In [11]:
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords, nltk.MLEProbDist)

##### Examples

In [12]:
# C('DET', 'NOUN'):
print("Frequency of C('DET', 'the') is",
      cfd_tagwords['DET']['the'])

# P('the' | 'DET')
print("Probability of P('the' | 'DET') is",
      cpd_tagwords['DET'].prob('the') )

Frequency of C('DET', 'the') is 62710
Probability of P('the' | 'DET') is 0.45767375327509324


### Task 1 (Joint probability)

Estimate the joint probability $P(t^{n}, w^{n})$ of the tag sequence `NOUN VERB VERB .` for the word sequence `birds can fly .` 
using the transition and emission probabilities estimated above.

Bonus: Implement the task in both raw probabilities and log probabilities.

#### Task 1: Code

In [13]:
# TODO: YOUR CODE HERE

#### Task 1: Solution

We solve the task using direct probabilities and also in the $\log$ space, which enables us to do summation instead of products (better numerical properties).

In [14]:
prob_tagsequence = \
  cpd_tags["START"].prob("NOUN")  * cpd_tagwords["NOUN"].prob("birds") * \
  cpd_tags["NOUN"].prob("VERB")   * cpd_tagwords["VERB"].prob("can") * \
  cpd_tags["VERB"].prob("VERB")   * cpd_tagwords["VERB"].prob("fly") * \
  cpd_tags["VERB"].prob(".")      * cpd_tagwords["."].prob(".") * \
  cpd_tags["."].prob("END")

md("The probability of the tag sequence `NOUN VERB VERB .` \
    for `birds can fly .` is:", prob_tagsequence)

The probability of the tag sequence `NOUN VERB VERB .`     for `birds can fly .` is: 6.5932971745681245e-15

In [15]:
log_prob_tagsequence = \
  np.log(cpd_tags["START"].prob("NOUN"))  + np.log(cpd_tagwords["NOUN"].prob("birds"))  +\
  np.log(cpd_tags["NOUN"].prob("VERB"))  + np.log(cpd_tagwords["VERB"].prob("can"))  +\
  np.log(cpd_tags["VERB"].prob("VERB"))  + np.log(cpd_tagwords["VERB"].prob("fly"))  + \
  np.log(cpd_tags["VERB"].prob("."))  + np.log(cpd_tagwords["."].prob("."))  + \
  np.log(cpd_tags["."].prob("END") )

md("The log probability of the tag sequence `NOUN VERB VERB .` \
    for `birds can fly .` is:", log_prob_tagsequence)
md("The probability of the tag sequence `NOUN VERB VERB .` \
    for `birds can fly .` is:", np.exp(log_prob_tagsequence))

The log probability of the tag sequence `NOUN VERB VERB .`     for `birds can fly .` is: -32.65272284153959

The probability of the tag sequence `NOUN VERB VERB .`     for `birds can fly .` is: 6.59329717456812e-15

### Task 2 (Viterbi algorithm)

Implement the Viterbi algorithm for PoS tagging.

Bonus: Implement the task in both raw probabilities and log probabilities.

#### Code

In [16]:
def viterbi(sentence, log_prob=False):
    # Step 1. Initialization step.
    # TODO: YOUR CODE HERE

    # Step 2. Recursion step.
    # TODO: YOUR CODE HERE

    # Step 3. Termination step.
    # TODO: YOUR CODE HERE

    # Step 4. Backtrace the path.
    # TODO: YOUR CODE HERE

    return best_tagsequence, prob_tagsequence

#### Solution

In [17]:
def viterbi(sentence, log_prob=False):
    # Step 1. Initialization step.
    sentlen = len(sentence)
    distinct_tags = np.array(list(set(all_tags)))
    tagslen = len(distinct_tags)

    # Initialize the viterbi and backpointer matrices.
    viterbi = np.zeros((tagslen, sentlen + 1), dtype=np.float64)
    backpointer = np.zeros((tagslen, sentlen + 1), dtype=np.uint32)
    
    for s, tag in enumerate(distinct_tags):
        if log_prob:
            viterbi[s, 0] =  np.log(cpd_tags["START"].prob(tag)) \
                + np.log(cpd_tagwords[tag].prob(sentence[0]))
        else:
            viterbi[s, 0] =  cpd_tags["START"].prob(tag) \
                * cpd_tagwords[tag].prob(sentence[0])

        backpointer[s, 0] = 0
    show_viterbi(viterbi, distinct_tags, sentence, t=0)
        

    # Step 2. Recursion step.
    for t in range(1, sentlen):
        for s, tag in enumerate(distinct_tags):
            current_viterbi = np.zeros(tagslen ,dtype=np.float64)
            for sprime, predtag in enumerate(distinct_tags):
                if log_prob:
                    current_viterbi[sprime] = viterbi[sprime, t - 1] + \
                                              np.log(cpd_tags[predtag].prob(tag)) + \
                                              np.log(cpd_tagwords[tag].prob(sentence[t]))
                else:
                    current_viterbi[sprime] = viterbi[sprime, t - 1] * \
                                              cpd_tags[predtag].prob(tag) * \
                                              cpd_tagwords[tag].prob(sentence[t])
            backpointer[s, t] = np.argmax(current_viterbi)
            viterbi[s, t] = max(current_viterbi)
        show_viterbi(viterbi, distinct_tags, sentence, t=t)


    # Step 3. Termination step.
    current_viterbi = np.empty(tagslen, dtype=np.float64)
    ind_of_end = -1
    for s, tag in enumerate(distinct_tags):
        if tag == "END":
            ind_of_end = s
        if log_prob:
            current_viterbi[s] = viterbi[s, sentlen - 1] + np.log(cpd_tags[tag].prob("END"))
        else:
            current_viterbi[s] = viterbi[s, sentlen - 1] * cpd_tags[tag].prob("END")

    backpointer[ind_of_end, sentlen] = np.argmax(current_viterbi)
    viterbi[ind_of_end, sentlen] = max(current_viterbi)
    show_viterbi(viterbi, distinct_tags, sentence, t=sentlen)

    # Step 4. Backtrace the path.
    best_tagsequence = []
    prob_tagsequence = viterbi[ind_of_end, sentlen]
    prev_ind  = ind_of_end
    for t in range(sentlen, 0, -1):
        prev_ind = backpointer[prev_ind, t]
        best_tagsequence.append(distinct_tags[prev_ind])
    best_tagsequence.reverse()

    return best_tagsequence, prob_tagsequence

### Task 3

Using your HMM, try to tag the sentences below. Then, answer the following questions:
* Why does it fail to predict tags for the sentence about ETH?
  * Could it just have mislabeled the sentence because it disagreed with the statement? ;-)

In [18]:
log_prob = False
input_sentence = "The dog is mine."
# input_sentence = "Have a nice day."
# input_sentence = "ETH is the best university in the world."

# Tag the sentence.
sentence =  nltk.word_tokenize(input_sentence)
best_tagsequence, prob_tagsequence = viterbi(sentence, log_prob=log_prob)

# Display the tags and the probability.
show_tags(zip(sentence, best_tagsequence))
prob_tagsequence = np.exp(prob_tagsequence) if log_prob else prob_tagsequence
print ("The probability of the best tag sequence is:", prob_tagsequence)

#### Viterbi probabilities at time $t = 0$

,The,dog,is,mine,.,
.,0,0,0,0,0,0
NOUN,0,0,0,0,0,0
END,0,0,0,0,0,0
DET,1.13e-02,0,0,0,0,0
START,0,0,0,0,0,0
PRON,0,0,0,0,0,0
VERB,0,0,0,0,0,0
PRT,0,0,0,0,0,0
ADP,0,0,0,0,0,0
NUM,0,0,0,0,0,0


#### Viterbi probabilities at time $t = 1$

,The,dog,is,mine,.,
.,0,0,0,0,0,0
NOUN,0,1.80e-06,0,0,0,0
END,0,0,0,0,0,0
DET,1.13e-02,0,0,0,0,0
START,0,0,0,0,0,0
PRON,0,0,0,0,0,0
VERB,0,0,0,0,0,0
PRT,0,0,0,0,0,0
ADP,0,0,0,0,0,0
NUM,0,0,0,0,0,0


#### Viterbi probabilities at time $t = 2$

...

,The,dog,is,mine,.,
.,0,0,0,0,2.18e-14,0
NOUN,0,1.80e-06,0,9.97e-14,0,0
END,0,0,0,0,0,0
DET,1.13e-02,0,0,1.86e-14,0,0
START,0,0,0,0,0,0
PRON,0,0,0,6.29e-13,0,0
VERB,0,0,1.57e-08,0,0,0
PRT,0,0,0,0,0,0
ADP,0,0,0,0,0,0
NUM,0,0,0,0,0,0


#### Viterbi probabilities at time $t = 5$

,The,dog,is,mine,.,
.,0,0,0,0,2.18e-14,0
NOUN,0,1.80e-06,0,9.97e-14,0,0
END,0,0,0,0,0,8.29e-15
DET,1.13e-02,0,0,1.86e-14,0,0
START,0,0,0,0,0,0
PRON,0,0,0,6.29e-13,0,0
VERB,0,0,1.57e-08,0,0,0
PRT,0,0,0,0,0,0
ADP,0,0,0,0,0,0
NUM,0,0,0,0,0,0


The,dog,is,mine,.
DET,NOUN,VERB,PRON,.


The probability of the best tag sequence is: 8.294025685357724e-15


#### Solution

It fails to label the sentence about ETH since ETH is an unknown word and its conditional probability is 0.
Check `cpd_tagwords["NOUN"].prob("ETH")`.
To work with unknown words, morphological analysis of words should be used.